# Images and Steering Angle Downloaded from Kaggle




In [ ]:

%%capture
!pip install kaggle
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# !kaggle datasets download -d shyamsundaravce/thesis
!unzip thesis -d images

# Import Libraries

In [ ]:
import cv2
import numpy as np;
import pandas as pd;
import tensorflow
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
    Lambda
)
from keras.optimizers import Adam;
from keras.losses import mse;
from keras.callbacks import EarlyStopping,ModelCheckpoint;
from sklearn.model_selection import train_test_split
import os
import random


In [ ]:
def load_img_steering(datadir, data):
    image_path = []
    steering = []
    for i in range(len(data)):
        index,lane_img,angle = data.iloc[i]
        image_path.append(os.path.join(datadir,lane_img))
        steering.append(angle)
    image_paths = np.asarray(image_path)
    steerings = np.asarray(steering)
    return image_paths, steerings

In [ ]:
imagedir =  r'/content/images'
csvdata = pd.read_csv(r'/content/SteerGroundTruth.csv');
dftown_7=csvdata[csvdata['A'].str.contains('Town07',case=False)]
image_paths, steerings = load_img_steering(imagedir, dftown_7)
X_train, X_valid, y_train, y_valid = train_test_split(
    image_paths, steerings, test_size=0.2, random_state=6
)
print("Training Samples: {}\nValid Samples: {}".format(len(X_train), len(X_valid)))

Training Samples: 9003
Valid Samples: 2251


In [ ]:
print(len(y_train))

9003


In [101]:
def img_preprocessing(img):
    cv_img = cv2.imread(img)
    roi_img = cv_img[235:, ]  # Region of Interest
    res_img = cv2.resize(roi_img, (200, 66), interpolation=cv2.INTER_AREA)  # Resizing the image
    return res_img


In [125]:
##Image Augmentation
## Each time , the steering angle , image gets moderated all at onec
def random_flip(image, steering_angle):
    image = cv2.flip(image, 1)
    steering_angle = -steering_angle
    return image, steering_angle


def translate(img,x,y):
    height,width = img.shape[0],img.shape[1]
    matrix = np.float32([[1,0,x],[0,1,0]])
    return cv2.warpAffine(img,matrix,(width,height))


# Theta value should be negative
def rotation(img,steering,angle):
    height,width = img.shape[0],img.shape[1]
    Matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    image = cv2.warpAffine(img,Matrix,(width,height))
    steering = -angle
    return image,steering


def augmentation(img,steering):
    """
    Currently I am doing only flipping and translation ,
    Because, I don't knew what will happen to rotation
    """
    range_x = 10
    range_y = 0
    image, steering_angle = random_flip(img, steering)
    image = translate(image, range_x, range_y)
    
    return image, steering_angle


In [126]:
#The keras will take care of the batchees 

def train_generator(images,steering,batch_size):
   
    for x in range(0,len(images),batch_size):
      batch_lane_img=[];
      batch_steering=[];
      batch_imagePath = images[x:x+batch_size]
      batch_steer = steering[x:x+batch_size]
      img = [img_preprocessing(x) for x in batch_imagePath]
      # Code FOR Data Augmentation 
      for x in range(len(batch_imagePath)):
        lane_img,lane_steer = augmentation(img[x],batch_steer[x])
        norm_img = lambda x: x / 127.5 - 1
        batch_lane_img.append(norm_img(lane_img))
        batch_steering.append(lane_steer)  
      yield np.asarray(batch_lane_img),np.asarray(batch_steering)  






In [127]:

def valid_generator(images,steering,batch_size):
    
    for x in range(0,len(images),batch_size):
      batch_lane_img=[];
      batch_imagePath = images[x:x+batch_size]
      batch_steer = steering[x:x+batch_size]
      img = [img_preprocessing(x) for x in batch_imagePath]
      
      # Code FOR Data Augmentation 
      for x in range(len(batch_imagePath)):
        norm_img = lambda x: x / 127.5 - 1
        batch_lane_img.append(norm_img(img[x]))
       
      yield np.asarray(batch_lane_img),np.asarray(batch_steer)  



# NVIDIA PilotNet Model

In [129]:

def PilotNet():
    
    model = Sequential()
    model.add(Conv2D(24,(5,5),strides=(2,2),input_shape=(66,200,3),activation="relu"))
    model.add(Conv2D(36,(5,5),strides=(2,2),activation="relu"))
    model.add(Conv2D(48,(5,5),strides=(2,2),activation="relu")) 
    model.add(Conv2D(64,(3,3),activation="relu"))   
    model.add(Conv2D(64,(3,3),activation="relu"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model


In [130]:

nvidia_model = PilotNet();
es = EarlyStopping(patience=15, restore_best_weights=True, monitor='val_accuracy')
checkpoint = ModelCheckpoint("Town07.h5", monitor='val_loss', mode='min', verbose=1, save_best_only=True)
nvidia_model.compile(loss=mse,metrics=['accuracy'],optimizer=Adam(lr=1e-3))


In [131]:
nvidia_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 31, 98, 24)        1824      
                                                                 
 conv2d_41 (Conv2D)          (None, 14, 47, 36)        21636     
                                                                 
 conv2d_42 (Conv2D)          (None, 5, 22, 48)         43248     
                                                                 
 conv2d_43 (Conv2D)          (None, 3, 20, 64)         27712     
                                                                 
 conv2d_44 (Conv2D)          (None, 1, 18, 64)         36928     
                                                                 
 dropout_32 (Dropout)        (None, 1, 18, 64)         0         
                                                                 
 flatten_8 (Flatten)         (None, 1152)             

In [ ]:
mini_batch = 32
nvidia_model.fit_generator(train_generator(X_train, y_train, 32),
                 epochs=100, 
                 steps_per_epoch=int(len(X_train)/mini_batch),
                 validation_data=valid_generator(X_valid, y_valid,32), 
                 validation_steps=int(len(X_valid)/mini_batch),
                 shuffle=False, 
                 callbacks=[checkpoint, es])

<ipython-input-132-21981b75238b>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  nvidia_model.fit_generator(train_generator(X_train, y_train, 32),


Epoch 1/100
140/281 [=============>................] - ETA: 29s - loss: 0.0067 - accuracy: 0.0627

In [ ]:
import csv

# create a list of integers


# open a CSV file for writing
with open('numbers.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # write each integer to a row in the CSV file
    for number in rac_count:
        writer.writerow([number])



In [36]:
set_rec = set(train_count)
print(len(set_rec))
print(len(train_count))

5730
9024


In [40]:
def generator():
  z=[]
  for x in range(0,100,25):
    y = [j for j in range(x,x+25)]
    z.append(y)
    yield z
for x in range(4):      
  re = next(generator())
  print(re)


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]


In [85]:
# def generator():
#     for x in range(0, 100, 25):
#         yield [j for j in range(x, x + 25)]

def generator( image_path, steering_angle, batch_size=25):
    for x in range(0,len(image_path),batch_size):
        batch_id   = image_path[x:x+batch_size]
        batch_steer = steering_angle[x:x+batch_size]
        batch_path= [x for x in batch_id];
        yield np.asarray(batch_path),np.asarray(batch_steer)
      
    

train_df = pd.read_csv('/content/SteerGroundTruth.csv')
train_df = train_df[0:100]
image_path=[]
steering_angle =[]
for x in range(100):
  index,A,B = train_df.iloc[x]
  image_path.append(A)
  steering_angle.append(B)

my_generator = generator(image_path,steering_angle,batch_size=25)

next(my_generator)


(array(['90003_Town01.jpg', '90004_Town01.jpg', '90005_Town01.jpg',
        '90006_Town01.jpg', '90007_Town01.jpg', '90008_Town01.jpg',
        '90009_Town01.jpg', '90010_Town01.jpg', '90011_Town01.jpg',
        '90012_Town01.jpg', '90013_Town01.jpg', '90014_Town01.jpg',
        '90015_Town01.jpg', '90016_Town01.jpg', '90017_Town01.jpg',
        '90018_Town01.jpg', '90019_Town01.jpg', '90020_Town01.jpg',
        '90021_Town01.jpg', '90022_Town01.jpg', '90023_Town01.jpg',
        '90024_Town01.jpg', '90025_Town01.jpg', '90026_Town01.jpg',
        '90027_Town01.jpg'], dtype='<U16'),
 array([-0.07653122, -0.05468272, -0.05468272, -0.05468272, -0.05468272,
        -0.05468272, -0.05468272, -0.05468272, -0.05468272, -0.05468272,
        -0.05468272, -0.05468272, -0.05468272, -0.05468272, -0.05468272,
        -0.05468272, -0.05468272, -0.05468272, -0.05468272, -0.05468272,
        -0.05468272, -0.05468272, -0.05069869, -0.05069869, -0.05069869]))

In [89]:
next(my_generator)

StopIteration: ignored

In [81]:
next(my_generator)

(array(['90054_Town01.jpg', '90055_Town01.jpg', '90056_Town01.jpg',
        '90057_Town01.jpg', '90058_Town01.jpg', '90060_Town01.jpg',
        '90061_Town01.jpg', '90063_Town01.jpg', '90064_Town01.jpg',
        '90065_Town01.jpg', '90066_Town01.jpg', '90067_Town01.jpg',
        '90068_Town01.jpg', '90069_Town01.jpg', '90070_Town01.jpg',
        '90071_Town01.jpg', '90072_Town01.jpg', '90073_Town01.jpg',
        '90074_Town01.jpg', '90075_Town01.jpg', '90076_Town01.jpg',
        '90077_Town01.jpg', '90078_Town01.jpg', '90080_Town01.jpg',
        '90081_Town01.jpg'], dtype='<U16'),
 array([-8.13172664e-06, -8.13172664e-06, -2.48524186e-04, -2.48524186e-04,
        -2.48524186e-04,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         6.15560450e-04,  6.15560450e-04,  6.15560450e-04,  6.15560450e-04,
         2.09874995e-02,  2.09874995e-02,  2.09874995e-02,  2.09874995e-02,
         1.58605501e-02,  1.5860

In [82]:
next(my_generator)

(array(['90082_Town01.jpg', '90083_Town01.jpg', '90084_Town01.jpg',
        '90085_Town01.jpg', '90086_Town01.jpg', '90087_Town01.jpg',
        '90088_Town01.jpg', '90089_Town01.jpg', '90090_Town01.jpg',
        '90091_Town01.jpg', '90092_Town01.jpg', '90094_Town01.jpg',
        '90095_Town01.jpg', '90096_Town01.jpg', '90097_Town01.jpg',
        '90098_Town01.jpg', '90099_Town01.jpg', '90100_Town01.jpg',
        '90101_Town01.jpg', '90103_Town01.jpg', '90104_Town01.jpg',
        '90105_Town01.jpg', '90106_Town01.jpg', '90108_Town01.jpg',
        '90109_Town01.jpg'], dtype='<U16'),
 array([ 0.0086865 ,  0.0086865 ,  0.0086865 ,  0.00301848,  0.00301848,
         0.00301848, -0.00013273, -0.00013273, -0.00013273, -0.00055572,
        -0.00055572, -0.00055572, -0.00055572,  0.00651448,  0.00651448,
         0.00651448,  0.00651448,  0.00515869,  0.00515869,  0.00515869,
         0.00332725,  0.00332725,  0.00332725,  0.00155591,  0.00155591]))

In [76]:
train_df

,index,A,B
0,0,90003_Town01.jpg,-0.076531
1,1,90004_Town01.jpg,-0.054683
2,2,90005_Town01.jpg,-0.054683
3,3,90006_Town01.jpg,-0.054683
4,4,90007_Town01.jpg,-0.054683
...,...,...,...
95,95,90104_Town01.jpg,0.003327
96,96,90105_Town01.jpg,0.003327
97,97,90106_Town01.jpg,0.003327
98,98,90108_Town01.jpg,0.001556
